In [2]:
import pandas as pd
import openpyxl
import numpy as np
from pathlib import Path
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import re
import os
import sys

In [1]:
import pmp_functions_v5 as pmp

In [4]:
# --- 1. DEFINE THE MISSING HELPER FUNCTION ---
def process_bloomberg_sheet(df, remove_header_rows=0):
    """
    Standard cleaning for Excel exports.
    1. Removes top N metadata rows.
    2. Finds the header row containing 'Date'.
    3. Resets index.
    """
    df = df.iloc[remove_header_rows:].copy()
    
    # Simple clean: Ensure we don't have empty rows at the top
    df = df.dropna(how='all')
    
    # If the first column isn't "Date", try to find the header
    # (This is a failsafe if headers are on row 3, for example)
    if "Date" not in df.columns:
        # Try to find the row that contains "Date"
        for i in range(min(10, len(df))):
            row_vals = df.iloc[i].astype(str).values
            if "Date" in row_vals:
                df.columns = df.iloc[i]
                df = df.iloc[i+1:]
                break
    
    return df


Loading Eq...
Loading Bd...
Loading Rt...
Loading FX...

--- RUNNING STRATEGY: EQUITY ---
   Assets: 7
   Realized Vol (Unscaled): 19.69%
   Scaling Factor: 0.51x

--- RUNNING STRATEGY: BOND ---
   Assets: 7
   Realized Vol (Unscaled): 9.80%
   Scaling Factor: 1.02x

--- RUNNING STRATEGY: RATES ---
   Assets: 7
   Realized Vol (Unscaled): 1.24%
   Scaling Factor: 3.00x

--- RUNNING STRATEGY: FX ---
   Assets: 6
   Realized Vol (Unscaled): 10.49%
   Scaling Factor: 0.95x

Final Portfolio Re-Scaling Factor: 2.00x

             PERFORMANCE SUMMARY PER ASSET CLASS       
                          Equity     Bond    Rates       FX  Total_Portfolio
Arithm Avg Total Return  -1.6811   1.2157   0.1295   0.3201          -0.0079
Arithm Avg Xs Return     -3.6811  -0.7843  -1.8705  -1.6799          -2.0079
Std Xs Returns           10.0000  10.0980   3.7181   9.2101          10.0000
Sharpe Arithmetic        -0.3681  -0.0777  -0.5031  -0.1824          -0.2008
Geom Avg Total Return    -2.1677   0.7085

In [22]:
# =============================================================================
# --- CONFIGURATION & CONSTANTS ---
# =============================================================================

# Paths & Targets
VOL_TARGET = 0.10
DATA_PATH = '../Data/'

# Strategy Parameters
MOMENTUM_LOOKBACKS = [1, 3, 12]  # Months for signal generation
SIGNAL_LAG = 1                   # Lag to apply to signal
K_SELECT = 1                     # Number of assets to select (CS Momentum)
MIN_ASSETS = 4                   # Min assets required to trade

# Risk Management & Costs
RF_ANNUAL = 0.02                 # Assumed risk-free rate (2%)
T_COST_BPS = 0.0000              # Transaction costs (10 basis points)
MAX_SCALAR_STRATEGY = 3.0        # Max leverage cap for individual asset class
MAX_SCALAR_PORTFOLIO = 2.0       # Max leverage cap for final aggregated portfolio

# Time & Frequency
ANNUALIZATION_FACTOR = 12
RESAMPLE_FREQ = "ME"             # Month End

# Derived Constants
RF_MONTHLY = RF_ANNUAL / ANNUALIZATION_FACTOR
T_COST = T_COST_BPS              # Passing directly as decimal if pmp expects decimal, or bps if expected bps

# =============================================================================
# 1. THE STRATEGY FACTORY
# =============================================================================
def run_independent_strategy(name, df_prices, benchmark, k_select=K_SELECT, min_assets=MIN_ASSETS):
    """
    Runs a complete, isolated Momentum L/S strategy for a single asset class.
    """
    print(f"\n--- RUNNING STRATEGY: {name} ---")
    print(f"   Assets: {len(df_prices.columns)}")
    
    # A. Prepare Data
    returns = df_prices.pct_change().fillna(0)
    rf_series = pd.Series(RF_MONTHLY, index=returns.index, name="RF")
    dummy_benchmark = returns.mean(axis=1) 
    
    # B. Signal Generation
    raw_signal = pd.DataFrame(0.0, index=df_prices.index, columns=df_prices.columns)
    
    for lag in MOMENTUM_LOOKBACKS:
        mom = df_prices.pct_change(lag)
        sig = np.sign(mom)
        raw_signal += sig
    
    final_signal = raw_signal / len(MOMENTUM_LOOKBACKS)
    
    # C. Weight Generation
    weights = pmp.make_country_weights(
        signal=final_signal,
        returns=returns,
        benchmark_series=dummy_benchmark,
        k=k_select,            
        long_short=True,
        beta_neutral=False,    
        signal_lag=SIGNAL_LAG,
        min_regions=min_assets 
    )
    
    # D. Preliminary Backtest (Unscaled)
    # We use 0.0 cost here to get pure realized volatility for scaling
    res_unscaled = pmp.run_cc_strategy(
        weights=weights,
        returns=returns,
        rf=rf_series,
        frequency=1,
        t_cost=0.0, 
        benchmark=benchmark
    )
    
    # E. Volatility Targeting
    ret_series = res_unscaled["ret_net"].dropna()
    realized_vol = ret_series.std() * np.sqrt(ANNUALIZATION_FACTOR)
    
    if realized_vol > 0:
        scalar = VOL_TARGET / realized_vol
        scalar = min(scalar, MAX_SCALAR_STRATEGY) 
    else:
        scalar = 1.0
        
    print(f"   Realized Vol (Unscaled): {realized_vol:.2%}")
    print(f"   Scaling Factor: {scalar:.2f}x")
    
    # F. Final Scaled Backtest
    weights_scaled = weights * scalar
    res_final = pmp.run_cc_strategy(
        weights=weights_scaled,
        returns=returns,
        rf=rf_series,
        frequency=1,
        t_cost=T_COST, 
        benchmark=benchmark
    )
    
    return res_final

# =============================================================================
# 2. DATA LOADING & PREPROCESSING
# =============================================================================

def load_and_process(filename, suffix, is_fx=False):
    """
    Helper to load and process data to avoid repetition.
    """
    print(f"Loading {suffix}...")
    full_path = os.path.join(DATA_PATH, filename)
    
    # Handle FX Returns vs Price data structure
    if is_fx:
        df = pd.read_excel(full_path, sheet_name="RETURNS", engine='openpyxl')
    else:
        df = pd.read_excel(full_path, engine='openpyxl')
        
    # Index setting
    if 'Date' in df.columns: df = df.set_index('Date')
    elif 'Dates' in df.columns: df = df.set_index('Dates')
    else: df = df.set_index(df.columns[0])
    
    df.index = pd.to_datetime(df.index, errors='coerce')
    
    # FX Special handling (cumprod of returns to get price index)
    if is_fx:
        df = (1 + df.fillna(0)).cumprod() * 100
    
    # Renaming and Resampling
    df.columns = [f"{c}_{suffix}" for c in df.columns]
    df = df.apply(pd.to_numeric, errors='coerce').resample(RESAMPLE_FREQ).last().ffill()
    return df

# --- Load Assets ---
df_eq = load_and_process("Equity Data.xlsx", "Eq")
df_bd = load_and_process("Bond Data.xlsx", "Bd")
df_rt = load_and_process("Interest Rates Data.xlsx", "Rt")
df_fx = load_and_process("FX Data.xlsx", "FX", is_fx=True)

# --- Load Benchmark ---
df_bench = pd.read_excel(os.path.join(DATA_PATH, "Benchmarks.xlsx"), engine='openpyxl')
if 'Date' in df_bench.columns: df_bench = df_bench.set_index('Date')
else: df_bench = df_bench.set_index(df_bench.columns[0])
df_bench.index = pd.to_datetime(df_bench.index, errors='coerce')
df_bench = df_bench.resample(RESAMPLE_FREQ).last().ffill()
benchmark_ret = df_bench['MSCI World'].pct_change().fillna(0)

# =============================================================================
# 3. EXECUTION PER ASSET CLASS
# =============================================================================

ret_equity = run_independent_strategy("EQUITY", df_eq, benchmark=benchmark_ret)
ret_bond = run_independent_strategy("BOND", df_bd, benchmark=benchmark_ret)
ret_rates = run_independent_strategy("RATES", df_rt, benchmark=benchmark_ret)
ret_fx = run_independent_strategy("FX", df_fx, benchmark=benchmark_ret)

# =============================================================================
# 4. AGGREGATION & PORTFOLIO CONSTRUCTION
# =============================================================================

# Combine Strategies
df_strategies = pd.DataFrame({
    'Equity': ret_equity["ret_net"],
    'Bond': ret_bond["ret_net"],
    'Rates': ret_rates["ret_net"],
    'FX': ret_fx["ret_net"]
}).dropna()

# Portfolio Construction (Equal Weight across strategies)
final_portfolio_ret = df_strategies.mean(axis=1)

# Re-Scale to Target Vol
port_vol = final_portfolio_ret.std() * np.sqrt(ANNUALIZATION_FACTOR)
final_scalar = VOL_TARGET / port_vol if port_vol > 0 else 1.0
final_scalar = min(final_scalar, MAX_SCALAR_PORTFOLIO)
final_portfolio_ret = final_portfolio_ret * final_scalar

print(f"\nFinal Portfolio Re-Scaling Factor: {final_scalar:.2f}x")

# Create Consolidated Dataframe for Summary
df_all = df_strategies.copy()
df_all['Total_Portfolio'] = final_portfolio_ret

# Align everything
common_idx = df_all.index.intersection(benchmark_ret.index)
df_all = df_all.loc[common_idx]
bm_aligned = benchmark_ret.loc[common_idx]
rf_aligned = pd.Series(RF_MONTHLY, index=common_idx)

# Calculate Excess Returns
xs_all = df_all.sub(rf_aligned, axis=0)
xs_bench = (bm_aligned - rf_aligned).to_frame()

# =============================================================================
# 5. SUMMARY
# =============================================================================

print("\n=======================================================")
print("             PERFORMANCE SUMMARY PER ASSET CLASS       ")
print("=======================================================")

summary = pmp.summarizePerformance(
    xsReturns=xs_all.values,
    Rf=rf_aligned.values.reshape(-1, 1),
    factorXsReturns=xs_bench.values, 
    annualizationFactor=ANNUALIZATION_FACTOR,
    strategyNames=df_all.columns.tolist()
)

print(summary)

# Correlation Matrix
print("\n--- Correlation Matrix ---")
print(df_all.corr().round(2))

# =============================================================================
# 6. VISUALIZATION
# =============================================================================
cum_all = (1 + df_all).cumprod()
cum_bench = (1 + bm_aligned).cumprod()

plt.figure(figsize=(12, 8))
colors = ['cyan', 'lightgreen', 'orange', 'pink']

# Plot Components
for i, col in enumerate(df_strategies.columns):
    plt.plot(cum_all[col], label=f"{col} L/S", alpha=0.5, linewidth=1.5, color=colors[i])

# Plot Total & Benchmark
plt.plot(cum_all['Total_Portfolio'], label=f"Total Portfolio ({VOL_TARGET:.0%} Vol)", color="blue", linewidth=3)
plt.plot(cum_bench, label="MSCI World", color="black", linestyle="--", linewidth=2, alpha=0.7)

plt.title("Multi-Asset Risk Parity: Component vs Aggregate Performance")
plt.ylabel("Cumulative Return (Log Scale)")
plt.yscale("log")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Equity Table

In [23]:
ret_equity

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US_Eq,w_AU_Eq,w_CH_Eq,w_JP_Eq,w_UK_Eq,w_EM_Eq,w_EU_Eq
Date,,,,,,,,,,,,,
1999-11-30,0.032103,0.032103,0.028167,0.000000,0.0,0.001667,-0.513383,0.000000,0.000000,0.545486,0.000000,0.00000,0.00000
1999-12-31,0.019049,0.019049,0.080969,1.037190,0.0,0.001667,0.534209,0.000000,-0.515160,0.000000,0.000000,0.00000,0.00000
2000-01-31,0.032031,0.032031,-0.057716,0.016928,0.0,0.001667,0.479277,0.000000,-0.447245,0.000000,0.000000,0.00000,0.00000
2000-02-29,-0.006877,-0.006877,0.002763,0.044495,0.0,0.001667,0.497246,0.000000,-0.504123,0.000000,0.000000,0.00000,0.00000
2000-03-31,0.008241,0.008241,0.069919,0.007072,0.0,0.001667,0.553629,0.000000,-0.545387,0.000000,0.000000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.001428,0.001428,0.059891,1.022695,0.0,0.001667,0.000000,-0.529875,0.000000,0.000000,0.531303,0.00000,0.00000
2025-06-30,-0.019857,-0.019857,0.043488,1.038345,0.0,0.001667,-0.532052,0.000000,0.512195,0.000000,0.000000,0.00000,0.00000
2025-07-31,-0.011014,-0.011014,0.013121,0.522124,0.0,0.001667,-0.517540,0.506526,0.000000,0.000000,0.000000,0.00000,0.00000


## Bond Table

In [24]:
ret_bond

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_EU_Bd,w_JP_Bd,w_AU_Bd,w_US_Bd,w_CH_Bd,w_EM_Bd,w_UK_Bd
Date,,,,,,,,,,,,,
1999-03-31,0.006896,0.006896,0.042831,0.000000,0.0,0.001667,-1.006519,0.000000,0.000000,0.0,1.013415,0.0,0.0
1999-04-30,0.021839,0.021839,0.039555,0.010729,0.0,0.001667,-1.011185,0.000000,0.000000,0.0,1.033024,0.0,0.0
1999-05-31,0.028392,0.028392,-0.034400,1.031615,0.0,0.001667,-0.997087,1.025479,0.000000,0.0,0.000000,0.0,0.0
1999-06-30,0.010371,0.010371,0.047652,0.014196,0.0,0.001667,-0.977846,0.988217,0.000000,0.0,0.000000,0.0,0.0
1999-07-31,0.012623,0.012623,-0.003216,1.015510,0.0,0.001667,0.000000,1.034245,-1.021622,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.026688,0.026688,0.043488,1.029443,0.0,0.001667,1.050500,-1.023812,0.000000,0.0,0.000000,0.0,0.0
2025-07-31,-0.008616,-0.008616,0.013121,1.037156,0.0,0.001667,0.000000,-1.013448,0.000000,0.0,1.004832,0.0,0.0
2025-08-31,0.027370,0.027370,0.026408,1.016388,0.0,0.001667,1.044417,-1.017046,0.000000,0.0,0.000000,0.0,0.0


## Rate Table

In [25]:
ret_rates

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_CH_Rt,w_EU_Rt,w_AU_Rt,w_US_Rt,w_EM_Rt,w_UK_Rt,w_JP_Rt
Date,,,,,,,,,,,,,
1999-04-30,0.008643,0.008643,0.039555,0.000000,0.0,0.001667,3.005173,0.000000,0.000000,-2.996531,0.000000,0.0000,0.000000
1999-05-31,0.004106,0.004106,-0.034400,0.004321,0.0,0.001667,2.999685,0.000000,0.000000,-2.995579,0.000000,0.0000,0.000000
1999-06-30,-0.004605,-0.004605,0.047652,0.002368,0.0,0.001667,2.996977,0.000000,0.000000,-3.001581,0.000000,0.0000,0.000000
1999-07-31,0.001395,0.001395,-0.003216,2.999279,0.0,0.001667,0.000000,0.000000,0.000000,-3.002212,0.000000,0.0000,3.003608
1999-08-31,0.002863,0.002863,-0.001388,3.002910,0.0,0.001667,0.000000,0.000000,2.998126,-2.995263,0.000000,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.000306,-0.000306,0.043488,6.044517,0.0,0.001667,-3.000000,2.999694,0.000000,0.000000,0.000000,0.0000,0.000000
2025-07-31,0.000000,0.000000,0.013121,2.999847,0.0,0.001667,-3.000000,0.000000,0.000000,0.000000,3.000000,0.0000,0.000000
2025-08-31,-0.000621,-0.000621,0.026408,3.000000,0.0,0.001667,-3.000000,0.000000,2.999379,0.000000,0.000000,0.0000,0.000000


## FX Table

In [26]:
ret_fx

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_CH_FX,w_EU_FX,w_JP_FX,w_UK_FX,w_AU_FX,w_EM_FX
Date,,,,,,,,,,,,
1990-03-31,0.052369,0.052369,0.000000,0.000000,0.0,0.001667,0.950367,0.000000,-0.897998,0.000000,0.0,0.000000
1990-04-30,0.038342,0.038342,0.000000,0.029295,0.0,0.001667,0.984928,0.000000,-0.946586,0.000000,0.0,0.000000
1990-05-31,-0.025295,-0.025295,0.000000,0.019171,0.0,0.001667,0.965886,0.000000,-0.991180,0.000000,0.0,0.000000
1990-06-30,0.011492,0.011492,0.000000,0.025056,0.0,0.001667,0.966136,0.000000,-0.954644,0.000000,0.0,0.000000
1990-07-31,0.008502,0.008502,0.000000,0.006913,0.0,0.001667,1.001869,0.000000,-0.993367,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.008906,0.008906,0.059891,1.960652,0.0,0.001667,-0.953765,0.000000,0.000000,0.962671,0.0,0.000000
2025-06-30,0.027808,0.027808,0.043488,1.911695,0.0,0.001667,0.985118,0.000000,0.000000,0.000000,0.0,-0.957311
2025-07-31,0.005708,0.005708,0.013121,0.971214,0.0,0.001667,0.927085,-0.921377,0.000000,0.000000,0.0,0.000000
